# Calculations for BBX behavioral data

This code notebook is intended to walkthrough the steps done for the various data calculations for the BBX behavioral dataset.  
Scoring is done based on the survery type, unique calculations can be found in the [data dict](https://docs.google.com/spreadsheets/d/15wvY8vC0pPF71aHHQauibh5d6ZQclbXh/edit#gid=1183265718) google sheet. Heosted here is also variable defnitions and reverse scoring values.  
We translate the *reverse scoring* variables before calculations.

## TOC:
* [Load Your Data](#dataload)
* [FFQ](#ffq)
* [BIS](#bis)



In [246]:
import os, glob
import sys
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [247]:
# uploading custom package 
sys.path.append('C:\\Users\\19802\\Documents\\nibl\\bbx\\code\\scripts')
import data_dict

## Load Your Data <a class="anchor" id="dataload"></a>


In [248]:
# Set data paths
data_path='C:\\Users\\19802\\Documents\\nibl'#'/Users/nikkibytes/Documents/git_nibl/bbx/data/behavioral/data_dict'

# Grab list of behavioral files found in the data bath (raw and clean data)
s1_behavioral=(os.path.join(data_path, "bbx_w1_msBxScn_01_06_21.tsv"))
scoring_file=os.path.join(data_path, 'bbx_msBxScn_datadict_12_30_20.xlsx')
ffq_nutrient_file=os.path.join(data_path, 'nutrient_data_by_ffq.tsv')



# Build dataframes from data_dictionary file, 
# scoring and conversion information found in these files
s1_behav=pd.read_csv(s1_behavioral, sep='\t')
s1_behav.set_index("patID", inplace=True)
nutrient_info=pd.read_csv(ffq_nutrient_file, sep="\t")
nutrient_info.columns = nutrient_info.iloc[0]
nutrient_info.set_index('FFQ_quest', inplace=True)

nutrient_info = nutrient_info[1:]
labels = pd.read_excel(scoring_file,  sheet_name="bbx_codebook")
labels = labels.loc[:,["LABEL", "NAME", "CODE", "range"]]
scoring = pd.read_excel(scoring_file, sheet_name="scoring")
scoring = scoring.loc[:,["Set", "Output", "Calculation","Description", "Notes"]]
reverse_scoring = pd.read_excel(scoring_file, sheet_name="reverse_scoring")






**View Dataframes:**

In [249]:
# Load behavioral data
display(s1_behav.head())
#print(s1_behav.columns.values)

,age,gender,w1Bx_ethnicity,w1Bx_race,w1Bx_date,w1Bx_height_1,w1Bx_weight_1,w1Bx_hungry,w1Bx_full,w1Bx_thirsty,w1Bx_nauseous,w1Bx_copleasant,w1Bx_codesirable,w1Bx_cosweet,w1Bx_cointense,w1Bx_cobitter,w1Bx_cofamiliar,w1Bx_ucopleasant,w1Bx_ucodesirable,w1Bx_ucosweet,w1Bx_ucointense,w1Bx_ucobitter,w1Bx_ucofamiliar,w1Bx_slpleasant,w1Bx_sldesirable,w1Bx_slsweet,w1Bx_slintense,w1Bx_slbitter,w1Bx_slfamiliar,w1Bx_uslpleasant,w1Bx_usldesirable,w1Bx_uslsweet,w1Bx_uslintense,w1Bx_uslbitter,w1Bx_uslfamiliar,w1Bx_diwpleasant,w1Bx_diwdesirable,w1Bx_diwsweet,w1Bx_diwintense,w1Bx_diwbitter,w1Bx_diwfamiliar,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76,w1Bx_bis_1,w1Bx_bis_2,w1Bx_bis_3,w1Bx_bis_4,w1Bx_bis_5,w1Bx_bis_6,w1Bx_bis_7,w1Bx_bis_8,w1Bx_bis_9,w1Bx_bis_10,w1Bx_bis_11,w1Bx_bis_12,w1Bx_bis_13,w1Bx_bis_14,w1Bx_bis_15,w1Bx_fciCrv_1,w1Bx_fciCrv_2,w1Bx_fciCrv_3,w1Bx_fciCrv_4,w1Bx_fciCrv_5,w1Bx_fciCrv_6,w1Bx_fciCrv_7,w1Bx_fciCrv_8,w1Bx_fciCrv_9,w1Bx_fciCrv_10,w1Bx_fciCrv_11,w1Bx_fciCrv_12,w1Bx_fciCrv_13,w1Bx_fciCrv_14,w1Bx_fciCrv_15,w1Bx_fciCrv_16,w1Bx_fciCrv_17,w1Bx_fciCrv_18,w1Bx_fciCrv_19,w1Bx_fciCrv_20,w1Bx_fciCrv_21,w1Bx_fciCrv_22,w1Bx_fciCrv_23,w1Bx_fciCrv_24,w1Bx_fciCrv_25,w1Bx_fciCrv_26,w1Bx_fciCrv_27,w1Bx_fciCrv_28,w1Bx_fciLik_1,w1Bx_fciLik_2,w1Bx_fciLik_3,w1Bx_fciLik_4,w1Bx_fciLik_5,w1Bx_fciLik_6,w1Bx_fciLik_7,w1Bx_fciLik_8,w1Bx_fciLik_9,w1Bx_fciLik_10,w1Bx_fciLik_11,w1Bx_fciLik_12,w1Bx_fciLik_13,w1Bx_fciLik_14,w1Bx_fciLik_15,w1Bx_fciLik_16,w1Bx_fciLik_17,w1Bx_fciLik_18,w1Bx_fciLik_19,w1Bx_fciLik_20,w1Bx_fciLik_21,w1Bx_fciLik_22,w1Bx_fciLik_23,w1Bx_fciLik_24,w1Bx_fciLik_25,w1Bx_fciLik_26,w1Bx_fciLik_27,w1Bx_fciLik_28,w1Bx_debq_1,w1Bx_debq_2,w1Bx_debq_3,w1Bx_debq_4,w1Bx_debq_5,w1Bx_debq_6,w1Bx_debq_7,w1Bx_debq_8,w1Bx_debq_9,w1Bx_debq_10,w1Bx_debq_11,w1Bx_debq_12,w1Bx_debq_13,w1Bx_debq_14,w1Bx_debq_15,w1Bx_debq_16,w1Bx_debq_17,w1Bx_debq_18,w1Bx_debq_19,w1Bx_debq_20,w1Bx_debq_21,w1Bx_debq_22,w1Bx_debq_23,w1Bx_debq_24,w1Bx_debq_25,w1Bx_debq_26,w1Bx_debq_27,w1Bx_debq_28,w1Bx_debq_29,w1Bx_debq_30,w1Bx_debq_31,w1Bx_debq_32,w1Bx_debq_33,w1Bx_ipaq_1,w1Bx_ipaq2_5,w1Bx_ipaq2_6,w1Bx_ipaq3_1,w1Bx_ipaq3_2,w1Bx_ipaq4_1,w1Bx_ipaq4_2,w1Bx_ipaq5_1,w1Bx_ipaq5_2,w1Bx_ipaq6_1,w1Bx_ipaq6_2,w1Bx_ipaq7_1,w1Bx_ipaq7_2,w1Bx_ipaq8_1,w1Bx_ipaq8_2,w1Bx_ipaq9_1,w1Bx_ipaq9_2,w1Bx_ipaq10_1,w1Bx_ipaq10_2,w1Bx_ipaq11_1,w1Bx_ipaq11_2,w1Bx_ipaq12_1,w1Bx_ipaq12_2,w1Bx_ipaq13_1,w1Bx_ipaq13_2,w1Bx_ipaq14_1,w1Bx_ipaq14_2,w1Bx_ipaq15_1,w1Bx_ipaq15_2,w1Bx_ipaq16_1,w1Bx_ipaq16_2,w1Bx_ipaq17_1,w1Bx_ipaq17_2,w1Bx_ipaq18_1,w1Bx_ipaq18_2,w1Bx_ipaq19_1,w1Bx_ipaq19_2,w1Bx_ipaq20_1,w1Bx_ipaq20_2,w1Bx_ipaq21_1,w1Bx_ipaq21_2,w1Bx_ipaq22_1,w1Bx_ipaq22_2,w1Bx_ipaq23_1,w1Bx_ipaq23_2,w1Bx_ipaq24_1,w1Bx_ipaq24_2,w1Bx_ipaq25_1,w1Bx_ipaq25_2,w1Bx_hand_1,w1Bx_hand_2,w1Bx_hand_3,w1Bx_hand_4,w1Bx_hand_5,w1Bx_hand_6,w1Bx_hand_7,w1Bx_hand_8,w1Bx_hand_9,w1Bx_hand_10,w1Bx_hand_11,w1Bx_hand_12,w1Bx_spsrq1,w1Bx_spsrq2,w1Bx_spsrq3,w1Bx_spsrq4,w1Bx_spsrq5,w1Bx_spsrq6,w1Bx_spsrq7,w1Bx_spsrq8,w1Bx_spsrq9,w1Bx_spsrq10,w1Bx_spsrq11,w1Bx_spsrq12,w1Bx_spsrq13,w1Bx_spsrq14,w1Bx_spsrq15,w1Bx_spsrq16,w1Bx_spsrq17,w1Bx_spsrq18,w1Bx_spsrq19,w1Bx_spsrq20,w1Bx_yfas_1,w1Bx_yfas

In [250]:
display(nutrient_info.head())
print(nutrient_info.columns.values)

,grams/med portion,kcal/100g,Pro/100g,fat(g)/100g,CHO/100g,sugars/100g,Satfat(g)/g
FFQ_quest,,,,,,,
ffq1,123.3,66.0,0.6,0.2,17.3,10.8,0.1
ffq2,84.8,47.5,0.8,0.2,11.9,10.1,0.1
ffq3,187.5,88.0,5.0,0.5,17.3,3.3,0.1
ffq4,119.0,91.0,5.4,1.2,15.3,0.5,0.4
ffq5,15.0,23.0,2.9,0.4,3.6,0.4,0.1


['grams/med portion ' 'kcal/100g' 'Pro/100g' 'fat(g)/100g' 'CHO/100g'
 'sugars/100g' 'Satfat(g)/g']


In [251]:
display(labels.head())
display(scoring.head())
display(reverse_scoring.head())

,LABEL,NAME,CODE,range
0,patID,Participant ID,NaN,NaN
1,age,Age (Years),NaN,NaN
2,gender,Gender,1 = Male\n2 = Female,1 to 2
3,w1Bx_ethnicity,Ethnicity (Hispanic or Non-Hispanic),"1 = Hispanic or Latino, 2= Non-Hispanic or Latino",1 to 2
4,w1Bx_race,Race,1=American Indian or Alaska Native\n2=Asian\n3=Black or African American\n4=Native Hawaiian or Other Pacific Islander\n5=White\n6=Middle Eastern or Northern African\n7=Other\n8 = Mixed,1 to 8


,Set,Output,Calculation,Description,Notes
0,FFQ,w1ffq##_kcal,w1Bx_ffq_## * kcal_ffq##,Total kcal consumed for each food variable,kcal_ffq## comes from FFQ excel sheet
1,FFQ,w1ffq_avgkcal,SUM(w1ffq##_kcal) / 14,Average kcal consumed each day for the reported 14 days,NaN
2,FFQ,w1ffq##_gfat,w1Bx_ffq_## * fat_ffq##,Total fat grams consumed for each food variable,fat_ffq## comes from FFQ excel sheet
3,FFQ,w1ffq##_fatkcal,SUM(w1ffq##_gfat) * 9,Total kcal derived from fat consumed for each food variable,NaN
4,FFQ,w1ffq_pctfat,SUM(w1ffq##_fatkcal)/SUM(w1ffq##_kcal)*100,Percent of total kcal derived from fat,NaN


,variable_name,old original score,new recoded score,new label
0,w1Bx_bis_6,4,1,Almost Always/Always
1,NaN,3,2,Often
2,NaN,2,3,Occasionally
3,NaN,1,4,Rarely/Never
4,w1Bx_bis_7,4,1,Almost Always/Always


In [252]:
#s1_behav.columns.values

## Calculations

In [253]:
df = s1_behav

In [310]:
"""
# Methods for Calculations 
"""
def ffq_calculations():
    ffq_dict={}
    s1_behav_copy=s1_behav

    # loop through the ffq## values(columns)
    for col_val in s1_behav.filter(like="ffq", axis=1).columns.values:
        #(col_val)

        """
        # Set Variables
        """    
        # extract ffq numerical id
        id_val=col_val.split("_")[2]
        # add id to dictionary


        # create string with numerical id to search in nutrient document
        ffq_id="ffq%s"%id_val
        #print("\n[INFO] getting %s calculations"%(ffq_id))

        # -- Get nutrient ffq averages --
        # get kcal_ffq##
        kcal_ffq=float(nutrient_info.loc[ffq_id, 'kcal/100g'])

        # get fat_ffq##
        fat_ffq=float(nutrient_info.loc[ffq_id, 'fat(g)/100g'])

        # get sug_ffq
        sug_ffq=float(nutrient_info.loc[ffq_id, 'sugars/100g'])

        # grams/med portion 
        grams_portion=float(nutrient_info.loc[ffq_id, 'grams/med portion '])

        # prepare the column -- translte into numeric to run calculations
        s1_behav_copy[col_val]=pd.to_numeric(s1_behav_copy[col_val], errors='coerce')
        z=np.asarray(s1_behav_copy[col_val])
        #print("\n[INFO] original data: \n", wBx_ffq[:10])

        for bbx_id in s1_behav.index.values:
            if bbx_id not in ffq_dict:
                ffq_dict[bbx_id]={}

            ffq_val=s1_behav_copy.loc[bbx_id,col_val]
            #print(type(ffq_val))


            """
            # Calculations
            """
            # -- Calculations --

            # --FFQ--

            # -- ffq##_kcal -- 
            w1ffq_kcal=ffq_val*kcal_ffq*(grams_portion/100)
            #print("\n[INFO] %s * %s"%(col_val, kcal_ffq))
            #print("[INFO] kcal data: \n", w1ffq_kcal[:10]

            # -- ffq##_gfat -- 
            w1ffq_gfat=ffq_val*fat_ffq
            #print("\n[INFO] %s * %s"%(col_val, fat_ffq))
            #print("[INFO] gfat data: \n", w1ffq_gfat)

            # -- ffq##_fatkcal -- 
            w1ffq_fatkcal=w1ffq_gfat*9
            #print("[INFO] fatkcal data: ", w1ffq_fatkcal)

            # -- ffq##_gsug -- 
            w1ffq_gsug=ffq_val*sug_ffq
            #print("\n[INFO] %s * %s"%(col_val, sug_ffq))
            #print("[INFO] gsug data: \n", w1ffq_gsug)

            # -- ffq##_sugkcal --
            w1ffq_sugkcal = w1ffq_gsug*4
            #print("[INFO] sugkcal data: ", w1ffq_sugkcal)

            # -- Fill Dictionary --
            ffq_dict[bbx_id]['w1Bx_ffq_%s_kcal'%id_val]=w1ffq_kcal
            ffq_dict[bbx_id]['w1Bx_ffq_%s_gfat'%id_val]=w1ffq_gfat
            ffq_dict[bbx_id]['w1Bx_ffq_%s_fatkcal'%id_val]=w1ffq_fatkcal
            ffq_dict[bbx_id]['w1Bx_ffq_%s_gsug'%id_val]=w1ffq_gsug 
            ffq_dict[bbx_id]['w1Bx_ffq_%s_sugkcal'%id_val]=w1ffq_sugkcal 


    # Create dataframe from dicionary
    ffq_df= pd.DataFrame(ffq_dict).T  
    # Filter dataframe into specific sets for calculations
    kcal_df=ffq_df.filter(like="kcal", axis=1)
    fatkcal_df=ffq_df.filter(like="fatkcal", axis=1)
    gsug_df=ffq_df.filter(like="gsug", axis=1)
    sugkcal_df=ffq_df.filter(like="sugkcal", axis=1)

    for bbx_id in s1_behav.index.values:
        """
        # Set Rows
        """
        kcal_row=kcal_df.loc[bbx_id,:]
        fatkcal_row=fatkcal_df.loc[bbx_id,:]  
        sugkcal_row=sugkcal_df.loc[bbx_id, :]

        """
        # Calculations 
        """
        # --avgkcal calculation--
        avgkcal=sum(kcal_row)/14

        # -- avgfatkcal -
        avgfatkcal = sum(fatkcal_row)/14

        # -- pctfat calculations --
        pctfat=(sum(fatkcal_row)/sum(kcal_row))*100
        pctfat_d = (avgfatkcal/avgkcal)*100

        # -- avgsugkcal calculation --
        avgsugkcal=sum(sugkcal_row)/14

        # -- pctsug --
        pctsug=(sum(sugkcal_row)/sum(kcal_row))*100



        """
        # Add to dictionary
        """
        # -- Fill Dictionary --
        ffq_dict[bbx_id]['w1ffq_avgkcal']=avgkcal
        ffq_dict[bbx_id]['w1ffq_avgfatkcal']=avgfatkcal
        ffq_dict[bbx_id]['w1ffq_avgsugkcal']=avgsugkcal
        ffq_dict[bbx_id]['w1ffq_pctfat']=pctfat
        ffq_dict[bbx_id]['w1ffq_pctfat_d']=pctfat_d

    
    # make dataframe 
    ffq_df_final= pd.DataFrame(ffq_dict).T  
    display(ffq_df_final.head())
    
    # wrote tp csv
    #path="C:\Users\19802\Documents\nibl\bbx_test_outputs"
    ffq_df_final.to_csv("w1Bx_ffq_calculations.csv")
    print('\n[INFO] completed FFQ calculations.\n')

### FFQ   <a anchor="class" id="ffq"></a>
---

In [311]:
# Get Calculations
ffq_calculations()

,w1Bx_ffq_1_kcal,w1Bx_ffq_1_gfat,w1Bx_ffq_1_fatkcal,w1Bx_ffq_1_gsug,w1Bx_ffq_1_sugkcal,w1Bx_ffq_2_kcal,w1Bx_ffq_2_gfat,w1Bx_ffq_2_fatkcal,w1Bx_ffq_2_gsug,w1Bx_ffq_2_sugkcal,w1Bx_ffq_3_kcal,w1Bx_ffq_3_gfat,w1Bx_ffq_3_fatkcal,w1Bx_ffq_3_gsug,w1Bx_ffq_3_sugkcal,w1Bx_ffq_4_kcal,w1Bx_ffq_4_gfat,w1Bx_ffq_4_fatkcal,w1Bx_ffq_4_gsug,w1Bx_ffq_4_sugkcal,w1Bx_ffq_5_kcal,w1Bx_ffq_5_gfat,w1Bx_ffq_5_fatkcal,w1Bx_ffq_5_gsug,w1Bx_ffq_5_sugkcal,w1Bx_ffq_6_kcal,w1Bx_ffq_6_gfat,w1Bx_ffq_6_fatkcal,w1Bx_ffq_6_gsug,w1Bx_ffq_6_sugkcal,w1Bx_ffq_7_kcal,w1Bx_ffq_7_gfat,w1Bx_ffq_7_fatkcal,w1Bx_ffq_7_gsug,w1Bx_ffq_7_sugkcal,w1Bx_ffq_8_kcal,w1Bx_ffq_8_gfat,w1Bx_ffq_8_fatkcal,w1Bx_ffq_8_gsug,w1Bx_ffq_8_sugkcal,w1Bx_ffq_9_kcal,w1Bx_ffq_9_gfat,w1Bx_ffq_9_fatkcal,w1Bx_ffq_9_gsug,w1Bx_ffq_9_sugkcal,w1Bx_ffq_10_kcal,w1Bx_ffq_10_gfat,w1Bx_ffq_10_fatkcal,w1Bx_ffq_10_gsug,w1Bx_ffq_10_sugkcal,w1Bx_ffq_11_kcal,w1Bx_ffq_11_gfat,w1Bx_ffq_11_fatkcal,w1Bx_ffq_11_gsug,w1Bx_ffq_11_sugkcal,w1Bx_ffq_12_kcal,w1Bx_ffq_12_gfat,w1Bx_ffq_12_fatkcal,w1Bx_ffq_12_gsug,w1Bx_ffq_12_sugkcal,w1Bx_ffq_13_kcal,w1Bx_ffq_13_gfat,w1Bx_ffq_13_fatkcal,w1Bx_ffq_13_gsug,w1Bx_ffq_13_sugkcal,w1Bx_ffq_14_kcal,w1Bx_ffq_14_gfat,w1Bx_ffq_14_fatkcal,w1Bx_ffq_14_gsug,w1Bx_ffq_14_sugkcal,w1Bx_ffq_15_kcal,w1Bx_ffq_15_gfat,w1Bx_ffq_15_fatkcal,w1Bx_ffq_15_gsug,w1Bx_ffq_15_sugkcal,w1Bx_ffq_16_kcal,w1Bx_ffq_16_gfat,w1Bx_ffq_16_fatkcal,w1Bx_ffq_16_gsug,w1Bx_ffq_16_sugkcal,w1Bx_ffq_17_kcal,w1Bx_ffq_17_gfat,w1Bx_ffq_17_fatkcal,w1Bx_ffq_17_gsug,w1Bx_ffq_17_sugkcal,w1Bx_ffq_18_kcal,w1Bx_ffq_18_gfat,w1Bx_ffq_18_fatkcal,w1Bx_ffq_18_gsug,w1Bx_ffq_18_sugkcal,w1Bx_ffq_19_kcal,w1Bx_ffq_19_gfat,w1Bx_ffq_19_fatkcal,w1Bx_ffq_19_gsug,w1Bx_ffq_19_sugkcal,w1Bx_ffq_20_kcal,w1Bx_ffq_20_gfat,w1Bx_ffq_20_fatkcal,w1Bx_ffq_20_gsug,w1Bx_ffq_20_sugkcal,w1Bx_ffq_21_kcal,w1Bx_ffq_21_gfat,w1Bx_ffq_21_fatkcal,w1Bx_ffq_21_gsug,w1Bx_ffq_21_sugkcal,w1Bx_ffq_22_kcal,w1Bx_ffq_22_gfat,w1Bx_ffq_22_fatkcal,w1Bx_ffq_22_gsug,w1Bx_ffq_22_sugkcal,w1Bx_ffq_23_kcal,w1Bx_ffq_23_gfat,w1Bx_ffq_23_fatkcal,w1Bx_ffq_23_gsug,w1Bx_ffq_23_sugkcal,w1Bx_ffq_24_kcal,w1Bx_ffq_24_gfat,w1Bx_ffq_24_fatkcal,w1Bx_ffq_24_gsug,w1Bx_ffq_24_sugkcal,w1Bx_ffq_25_kcal,w1Bx_ffq_25_gfat,w1Bx_ffq_25_fatkcal,w1Bx_ffq_25_gsug,w1Bx_ffq_25_sugkcal,w1Bx_ffq_26_kcal,w1Bx_ffq_26_gfat,w1Bx_ffq_26_fatkcal,w1Bx_ffq_26_gsug,w1Bx_ffq_26_sugkcal,w1Bx_ffq_27_kcal,w1Bx_ffq_27_gfat,w1Bx_ffq_27_fatkcal,w1Bx_ffq_27_gsug,w1Bx_ffq_27_sugkcal,w1Bx_ffq_28_kcal,w1Bx_ffq_28_gfat,w1Bx_ffq_28_fatkcal,w1Bx_ffq_28_gsug,w1Bx_ffq_28_sugkcal,w1Bx_ffq_29_kcal,w1Bx_ffq_29_gfat,w1Bx_ffq_29_fatkcal,w1Bx_ffq_29_gsug,w1Bx_ffq_29_sugkcal,w1Bx_ffq_30_kcal,w1Bx_ffq_30_gfat,w1Bx_ffq_30_fatkcal,w1Bx_ffq_30_gsug,w1Bx_ffq_30_sugkcal,w1Bx_ffq_31_kcal,w1Bx_ffq_31_gfat,w1Bx_ffq_31_fatkcal,w1Bx_ffq_31_gsug,w1Bx_ffq_31_sugkcal,w1Bx_ffq_32_kcal,w1Bx_ffq_32_gfat,w1Bx_ffq_32_fatkcal,w1Bx_ffq_32_gsug,w1Bx_ffq_32_sugkcal,w1Bx_ffq_33_kcal,w1Bx_ffq_33_gfat,w1Bx_ffq_33_fatkcal,w1Bx_ffq_33_gsug,w1Bx_ffq_33_sugkcal,w1Bx_ffq_34_kcal,w1Bx_ffq_34_gfat,w1Bx_ffq_34_fatkcal,w1Bx_ffq_34_gsug,w1Bx_ffq_34_sugkcal,w1Bx_ffq_35_kcal,w1Bx_ffq_35_gfat,w1Bx_ffq_35_fatkcal,w1Bx_ffq_35_gsug,w1Bx_ffq_35_sugkcal,w1Bx_ffq_36_kcal,w1Bx_ffq_36_gfat,w1Bx_ffq_36_fatkcal,w1Bx_ffq_36_gsug,w1Bx_ffq_36_sugkcal,w1Bx_ffq_37_kcal,w1Bx_ffq_37_gfat,w1Bx_ffq_37_fatkcal,w1Bx_ffq_37_gsug,w1Bx_ffq_37_sugkcal,w1Bx_ffq_38_kcal,w1Bx_ffq_38_gfat,w1Bx_ffq_38_fatkcal,w1Bx_ffq_38_gsug,w1Bx_ffq_38_sugkcal,w1Bx_ffq_39_kcal,w1Bx_ffq_39_gfat,w1Bx_ffq_39_fatkcal,w1Bx_ffq_39_gsug,w1Bx_ffq_39_sugkcal,w1Bx_ffq_40_kcal,w1Bx_ffq_40_gfat,w1Bx_ffq_40_fatkcal,w1Bx_ffq_40_gsug,w1Bx_ffq_40_sugkcal,w1Bx_ffq_41_kcal,w1Bx_ffq_41_gfat,w1Bx_ffq_41_fatkcal,w1Bx_ffq_41_gsug,w1Bx_ffq_41_sugkcal,w1Bx_ffq_42_kcal,w1Bx_ffq_42_gfat,w1Bx_ffq_42_fatkcal,w1Bx_ffq_42_gsug,w1Bx_ffq_42_sugkcal,w1Bx_ffq_43_kcal,w1Bx_ffq_43_gfat,w1Bx_ffq_43_fatkcal,w1Bx_ffq_43_gsug,w1Bx_ffq_43_sugkcal,w1Bx_ffq_44_kcal,w1Bx_ffq_44_gfat,w1Bx_ffq_44_fatkcal,w1Bx_ffq_44_gsug,w1Bx_ffq_44_sugkcal,w1Bx_ffq_45_kcal,w1Bx_ffq_45_gfat,w1Bx_f


[INFO] completed FFQ calculations.



---

In [254]:
# FFQ Calculations
scoring.loc[scoring['Set']=="FFQ"]

,Set,Output,Calculation,Description,Notes
0,FFQ,w1ffq##_kcal,w1Bx_ffq_## * kcal_ffq##,Total kcal consumed for each food variable,kcal_ffq## comes from FFQ excel sheet
1,FFQ,w1ffq_avgkcal,SUM(w1ffq##_kcal) / 14,Average kcal consumed each day for the reported 14 days,NaN
2,FFQ,w1ffq##_gfat,w1Bx_ffq_## * fat_ffq##,Total fat grams consumed for each food variable,fat_ffq## comes from FFQ excel sheet
3,FFQ,w1ffq##_fatkcal,SUM(w1ffq##_gfat) * 9,Total kcal derived from fat consumed for each food variable,NaN
4,FFQ,w1ffq_pctfat,SUM(w1ffq##_fatkcal)/SUM(w1ffq##_kcal)*100,Percent of total kcal derived from fat,NaN
5,FFQ,w1ffq##_gsug,w1Bx_ffq_## * sug_ffq##,Total sugar grams consumed for each food variable,sug_ffq## comes from FFQ excel sheet
6,FFQ,w1ffq##_sugkcal,SUM(w1ffq##_gsug) * 4,Total kcal derived from sugar consumed for each food variable,NaN
7,FFQ,w1ffq_pctsug,SUM(w1ffq##_sugkcal)/SUM(w1ffq##_kcal)*100,Percent of total kcal derived from sugar,NaN


In [300]:
# Look at the filtered df
s1_behav.filter(like="ffq", axis=1).head()

,w1Bx_ffq_1,w1Bx_ffq_2,w1Bx_ffq_3,w1Bx_ffq_4,w1Bx_ffq_5,w1Bx_ffq_6,w1Bx_ffq_7,w1Bx_ffq_8,w1Bx_ffq_9,w1Bx_ffq_10,w1Bx_ffq_11,w1Bx_ffq_12,w1Bx_ffq_13,w1Bx_ffq_14,w1Bx_ffq_15,w1Bx_ffq_16,w1Bx_ffq_17,w1Bx_ffq_18,w1Bx_ffq_19,w1Bx_ffq_20,w1Bx_ffq_21,w1Bx_ffq_22,w1Bx_ffq_23,w1Bx_ffq_24,w1Bx_ffq_25,w1Bx_ffq_26,w1Bx_ffq_27,w1Bx_ffq_28,w1Bx_ffq_29,w1Bx_ffq_30,w1Bx_ffq_31,w1Bx_ffq_32,w1Bx_ffq_33,w1Bx_ffq_34,w1Bx_ffq_35,w1Bx_ffq_36,w1Bx_ffq_37,w1Bx_ffq_38,w1Bx_ffq_39,w1Bx_ffq_40,w1Bx_ffq_41,w1Bx_ffq_42,w1Bx_ffq_43,w1Bx_ffq_44,w1Bx_ffq_45,w1Bx_ffq_46,w1Bx_ffq_47,w1Bx_ffq_48,w1Bx_ffq_49,w1Bx_ffq_50,w1Bx_ffq_51,w1Bx_ffq_52,w1Bx_ffq_53,w1Bx_ffq_54,w1Bx_ffq_55,w1Bx_ffq_56,w1Bx_ffq_57,w1Bx_ffq_58,w1Bx_ffq_59,w1Bx_ffq_60,w1Bx_ffq_61,w1Bx_ffq_62,w1Bx_ffq_63,w1Bx_ffq_64,w1Bx_ffq_65,w1Bx_ffq_66,w1Bx_ffq_67,w1Bx_ffq_68,w1Bx_ffq_69,w1Bx_ffq_70,w1Bx_ffq_71,w1Bx_ffq_72,w1Bx_ffq_73,w1Bx_ffq_74,w1Bx_ffq_75,w1Bx_ffq_76
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,5.0,5.0,1.0,1.0,3.0,3,1,3,4,3,2,3,3,1,1,3,2,1,2,1,1,2,2,1,2,1,1,2,1,1,1,1,2,2,1,2,2,1,2,1,2,1,1,1,1,2,1,1,1,2,4,2,2,1,2,2,1,1,1,1,1,1,1,2,2,1,1,1,1,1,1,5,1,1,4,1
bbx_002,5.0,6.0,2.0,1.0,1.0,4,4,2,3,3,2,3,3,1,2,2,2,1,1,1,2,3,2,1,1,2,1,1,1,2,1,1,1,1,1,1,1,2,2,1,2,1,1,1,2,1,3,1,1,1,3,2,1,2,2,2,1,1,1,1,1,1,2,2,1,1,1,1,1,2,1,1,4,1,1,1
bbx_003,6.0,6.0,5.0,1.0,5.0,4,1,3,2,3,1,5,2,1,3,4,3,1,1,1,2,4,1,1,4,2,1,4,1,2,2,1,1,2,1,2,1,3,3,1,3,3,4,1,6,3,3,3,2,5,3,2,3,2,2,1,6,3,1,1,1,1,3,1,3,1,2,1,1,1,1,2,1,1,3,1
bbx_004,2.0,2.0,2.0,1.0,3.0,4,4,2,2,3,3,2,2,2,1,2,2,1,1,1,1,5,2,1,1,2,2,2,2,2,1,1,2,1,1,1,1,1,3,1,1,1,1,2,3,2,1,2,1,1,2,1,3,1,3,2,1,1,2,1,1,1,1,5,3,1,1,1,1,1,1,4,6,1,1,2
bbx_005,3.0,2.0,1.0,1.0,2.0,2,3,1,2,2,1,2,3,2,2,3,3,2,1,1,3,3,2,1,2,1,1,2,1,2,1,2,2,2,1,1,1,3,2,1,3,2,1,2,2,2,1,1,1,1,2,1,2,1,4,2,3,1,1,1,1,1,3,1,4,1,2,1,3,1,1,4,4,2,1,1


## BIS <a anchor="class" id="bis"></a>
--- 

In [301]:
# FFQ Calculations
scoring.loc[scoring['Set']=="BIS-15"]

,Set,Output,Calculation,Description,Notes
8,BIS-15,w1bis_tot,SUM(w1Bx_bis_##),Total score on BIS-15,* assuming reverse scoring completed
9,BIS-15,w1bis_npi,"SUM(w1Bx_bis_6, w1Bx_bis_7, w1Bx_bis_8, w1Bx_bis_9, w1Bx_bis_10)",Score on Non-Planning Impulsivity subscale of BIS-15,* assuming reverse scoring completed
10,BIS-15,w1bis_mi,"SUM(w1Bx_bis_1, w1Bx_bis_2, w1Bx_bis_3, w1Bx_bis_4, w1Bx_bis_5)",Score on Motor Impulsivity subscale of BIS-15,* assuming reverse scoring completed
11,BIS-15,w1bis_ai,"SUM(w1Bx_bis_11, w1Bx_bis_12, w1Bx_bis_13, w1Bx_bis_14, w1Bx_bis_15)",Score on Attentional Impulsivity subscale of BIS-15,* assuming reverse scoring completed


In [288]:
# Look at the filtered df
s1_behav.filter(like="bis", axis=1).head(10)

,w1Bx_bis_1,w1Bx_bis_2,w1Bx_bis_3,w1Bx_bis_4,w1Bx_bis_5,w1Bx_bis_6,w1Bx_bis_7,w1Bx_bis_8,w1Bx_bis_9,w1Bx_bis_10,w1Bx_bis_11,w1Bx_bis_12,w1Bx_bis_13,w1Bx_bis_14,w1Bx_bis_15,w1Bx_bis_bas_1,w1Bx_bis_bas_2,w1Bx_bis_bas_3,w1Bx_bis_bas_4,w1Bx_bis_bas_5,w1Bx_bis_bas_6,w1Bx_bis_bas_7,w1Bx_bis_bas_8,w1Bx_bis_bas_9,w1Bx_bis_bas_10,w1Bx_bis_bas_11,w1Bx_bis_bas_12,w1Bx_bis_bas_13,w1Bx_bis_bas_14,w1Bx_bis_bas_15,w1Bx_bis_bas_16,w1Bx_bis_bas_17,w1Bx_bis_bas_18,w1Bx_bis_bas_19,w1Bx_bis_bas_20,w1Bx_bis_bas_21,w1Bx_bis_bas_22,w1Bx_bis_bas_23
patID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bbx_001,2,2,1,1,2,4,4,3,3,3,2,1,2,2,1,1,2,2,2,2,1,2,2,2,2,2,2,2,3,3,2,2,2,2,3,2,2,2
bbx_002,3,3,2,3,2,3,4,4,3,3,2,2,2,1,1,1,4,3,1,1,1,2,3,3,3,2,2,1,2,3,1,2,2,2,3,3,1,2
bbx_003,2,1,1,1,1,3,4,4,4,3,3,2,2,1,2,1,4,2,1,1,1,4,1,1,1,2,2,2,2,1,1,2,1,2,2,2,1,2
bbx_004,1,2,1,1,1,4,4,4,4,4,2,2,3,2,2,2,3,1,1,1,1,2,1,1,4,2,2,2,3,3,1,2,2,2,2,1,1,2
bbx_005,2,2,2,1,1,4,4,4,3,3,2,1,2,3,3,2,2,2,1,2,1,3,1,3,1,2,2,1,3,1,2,1,1,2,3,1,2,2
bbx_006,2,2,1,1,2,3,4,3,4,4,4,2,1,2,2,1,3,1,1,1,1,1,1,2,1,2,1,1,2,2,2,1,1,1,2,1,1,1
bbx_007,2,2,1,2,2,4,4,3,3,3,2,2,3,2,2,1,3,1,2,1,1,2,2,1,3,2,2,2,2,2,2,2,2,1,3,2,1,2
bbx_008,3,2,2,2,3,4,4,4,2,2,2,2,2,2,1,1,3,2,1,1,1,2,2,2,1,2,1,1,3,2,1,2,2,2,2,1,2,2
bbx_009,4,1,1,1,2,4,4,4,4,4,2,1,3,1,2,1,4,2,1,2,1,1,2,2,2,3,1,2,3,2,2,2,1,2,3,3,1,1


In [ ]:
# get rid of leading, irrelevant rows  
# WARNING: run ONLY ONCE
s1_raw_behav.drop([0,1,2,3,4,5,6], inplace=True)

In [187]:
print("[INFO] raw behavioral file (note the dropped rows): ")
display(s1_raw_behav.head())

[INFO] raw behavioral file (note the dropped rows): 


,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,SC6,SC7,SC8,SC9,SC10,SC11,SC12,SC13,SC14,BMI
7,2018-03-19 07:44:07,2018-03-19 09:22:59,IP Address,152.23.86.81,100,5931,True,2018-03-19 09:23:01,R_2SrhSBMJXhLwPbo,NaN,...,29,24,11,11,16,18,33,32,134,21.07869282404572
8,2018-03-19 12:15:21,2018-03-19 13:35:15,IP Address,152.23.86.81,100,4793,True,2018-03-19 13:35:16,R_2wvooa4ew57SG93,NaN,...,21,22,9,12,19,21,28,21,111,34.55075679991884
9,2018-03-23 13:40:24,2018-03-23 14:58:57,IP Address,152.23.213.32,100,4711,True,2018-03-23 14:58:58,R_2b3sEMOtTJmktVl,NaN,...,35,22,12,10,19,19,36,37,143,22.85640495867769
10,2018-03-23 15:04:04,2018-03-23 16:20:11,IP Address,152.23.213.32,98,4566,False,2018-03-24 11:54:33,R_4JdBh5aSkJ67kPL,NaN,...,34,23,14,13,18,18,18,34,113,20.37629412663135
11,2018-03-28 10:16:59,2018-03-28 11:40:50,IP Address,152.23.86.81,100,5031,True,2018-03-28 11:40:52,R_2dnKdQ1rXBZaTAA,NaN,...,28,24,14,14,20,23,32,38,162,21.77145582582835


In [188]:
# We find columns we don't want to keep by finding what columns are not 
# in the clean excel reference file
# -note we are keeping BMI and Recorded date, unlike the reference file

drop_list = [ 'Status', 'IPAddress', 'ResponseId', 'RecipientLastName',
             'RecipientFirstName', 'RecipientEmail', 'ExternalReference',
             'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 
             'UserLanguage']
print('\n[INFO] dropping these columns from the behavioral file. \n\n',drop_list)


[INFO] dropping these columns from the behavioral file. 

 ['Status', 'IPAddress', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage']


In [189]:
# drop columns
s1_raw_behav.drop(drop_list, axis=1, inplace=True)


In [190]:
print("[INFO] current behavioral dataframe:")
display(s1_raw_behav.head())

[INFO] current behavioral dataframe:


,StartDate,EndDate,Progress,Duration (in seconds),Finished,RecordedDate,w1behav_assessor,participantID,w1behav_consent,w1behav_height_1,...,SC6,SC7,SC8,SC9,SC10,SC11,SC12,SC13,SC14,BMI
7,2018-03-19 07:44:07,2018-03-19 09:22:59,100,5931,True,2018-03-19 09:23:01,Gandee bauert,BBX_001,Yes,155.7,...,29,24,11,11,16,18,33,32,134,21.07869282404572
8,2018-03-19 12:15:21,2018-03-19 13:35:15,100,4793,True,2018-03-19 13:35:16,Gandee jones,BBX_002,Yes,167.9,...,21,22,9,12,19,21,28,21,111,34.55075679991884
9,2018-03-23 13:40:24,2018-03-23 14:58:57,100,4711,True,2018-03-23 14:58:58,Gandee,bbx_005,Yes,176.0,...,35,22,12,10,19,19,36,37,143,22.85640495867769
10,2018-03-23 15:04:04,2018-03-23 16:20:11,98,4566,False,2018-03-24 11:54:33,Gandee,bbx_004,Yes,167.4,...,34,23,14,13,18,18,18,34,113,20.37629412663135
11,2018-03-28 10:16:59,2018-03-28 11:40:50,100,5031,True,2018-03-28 11:40:52,Bauert,BBX 006,Yes,163.5,...,28,24,14,14,20,23,32,38,162,21.77145582582835


In [191]:
#list(s1_raw_behav.keys())


## Data Translations <a anchor="class" id="datatrans"></a>

### Data Numerical Translation 
Converting the string data found in the behavioral to their corresponding numerical value.  

In [192]:
ffq_df = s1_raw_behav.filter(regex="ffq", axis=1)

bis_df = s1_raw_behav.filter(regex="bis", axis=1)

fci_df = s1_raw_behav.filter(regex="fci", axis=1)

debq_df = s1_raw_behav.filter(regex="debq", axis=1)

ipaq_df = s1_raw_behav.filter(regex="ipaq", axis=1)

hand_df = s1_raw_behav.filter(regex="hand", axis=1)

spsrq_df = s1_raw_behav.filter(regex="spsrg", axis=1)

yfas_df = s1_raw_behav.filter(regex="yfas", axis=1)

bisbas_df = s1_raw_behav.filter(regex="bisbas", axis=1)

pfs_df = s1_raw_behav.filter(regex="pfs", axis=1)



In [193]:
#display(var_df)
new_name = ''
for x in var_df["Variable Name"]:
    #print(x, new_name)
    if type(x) == str:
        #print('X: ', x)
        y = x.split("_")[1]
        #print('Y: ', y)
        var_df["Variable Name"].replace(x, y, inplace=True)

print('[INFO] updated version: ', var_df)

[INFO] updated version:     Variable Name Range                   Format Label
0      ethnicity     0     0 = Non-Hispanic or Latino
1            NaN     1          1= Hispanic or Latino
2           race     1                      1 = White
3            NaN     2  2 = Black or African American
4            NaN     3                      3 = Asian
..           ...   ...                            ...
74         sleep     1                  1 = Very good
75           NaN     2              2 = Somewhat good
76           NaN     3                    3 = Average
77           NaN     4              4 = Somewhat poor
78           NaN     5                  5 = Very poor

[79 rows x 3 columns]


In [194]:
#display(var_df)

for index, value in var_df["Variable Name"].iteritems():
    #print(index, value)
    if type(value) == str:
        new_name=value
        #print(new_name)
        
    else:
        #print("renaming {} to {}".format(value, new_name))
        #print(cookbook_df.loc[index, "Variable Name"])
        var_df.loc[index, "Variable Name"] = new_name
print('[INFO] final updated version: ', var_df)

[INFO] final updated version:     Variable Name Range                   Format Label
0      ethnicity     0     0 = Non-Hispanic or Latino
1      ethnicity     1          1= Hispanic or Latino
2           race     1                      1 = White
3           race     2  2 = Black or African American
4           race     3                      3 = Asian
..           ...   ...                            ...
74         sleep     1                  1 = Very good
75         sleep     2              2 = Somewhat good
76         sleep     3                    3 = Average
77         sleep     4              4 = Somewhat poor
78         sleep     5                  5 = Very poor

[79 rows x 3 columns]


In [195]:
var_df["Variable Name"].unique()


array(['ethnicity', 'race', 'gender', 'ffq##', 'bis##', 'fci#1', 'fci#2',
       'debq##', 'ipaq##', 'hand##', 'spsrq##', 'yfas', 'yfas17',
       'bisbas##', 'pfs##', 'hrsfasting', 'sex', 'sleep'], dtype=object)

In [196]:
fci1_code= var_df.loc[var_df["Variable Name"] == 'fci#1' ]
display('[INFO] fci1: ', fci1_code)
fci2_code= var_df.loc[var_df["Variable Name"] == 'fci#2' ]
display('[INFO] fci2: ', fci2_code)

display('[INFO] original: ', fci_df.head())

for val in fci1_code["Format Label"]:
    code_str = val.split("=")[1].lstrip(" ")
    code_range = val.split("=")[0].strip(" ")
    #print(code_str ,code_range)
    fci_df.replace(code_str, code_range, inplace=True)
    

for val in fci2_code["Format Label"]:
    code_str = val.split("=")[1].lstrip(" ")
    code_range = val.split("=")[0].strip(" ")
    #print(code_str ,code_range)
    fci_df.replace(code_str, code_range, inplace=True)  

display('[INFO] updated version: ', fci_df.head())

'[INFO] fci1: '

,Variable Name,Range,Format Label
22,fci#1,1,1 = Never
23,fci#1,2,2 = Rarely (once or twice)
24,fci#1,3,3 = Sometimes
25,fci#1,4,4 = Often
26,fci#1,5,5 = Always/Almost Every Day


'[INFO] fci2: '

,Variable Name,Range,Format Label
27,fci#2,1,1 = Dislike
28,fci#2,2,2 = Like a Little
29,fci#2,3,3 = Like a Lot
30,fci#2,4,4 = Love


'[INFO] original: '

,w1behav_fci#1_1,w1behav_fci#1_2,w1behav_fci#1_3,w1behav_fci#1_4,w1behav_fci#1_5,w1behav_fci#1_6,w1behav_fci#1_7,w1behav_fci#1_8,w1behav_fci#1_9,w1behav_fci#1_10,...,w1behav_fci#2_19,w1behav_fci#2_20,w1behav_fci#2_21,w1behav_fci#2_22,w1behav_fci#2_23,w1behav_fci#2_24,w1behav_fci#2_25,w1behav_fci#2_26,w1behav_fci#2_27,w1behav_fci#2_28
7,Rarely (once or twice),Sometimes,Sometimes,Never,Rarely (once or twice),Sometimes,Sometimes,Never,Rarely (once or twice),Sometimes,...,Like a Little,Like a Little,Like a Little,Like a Little,Like a Little,Like a Little,Like a Little,Love,Like a Little,Dislike
8,Sometimes,Sometimes,Never,Never,Never,Never,Never,Rarely (once or twice),Never,Rarely (once or twice),...,Like a Lot,Dislike,Like a Lot,Dislike,Dislike,Dislike,Like a Lot,Like a Lot,Like a Lot,Like a Little
9,Sometimes,Sometimes,Never,Never,Never,Never,Sometimes,Sometimes,Rarely (once or twice),Never,...,Dislike,Like a Little,Like a Lot,Dislike,Love,Like a Lot,Love,Like a Little,Like a Lot,Like a Little
10,Sometimes,Sometimes,Never,Never,Never,Never,Rarely (once or twice),Rarely (once or twice),Never,Never,...,Like a Little,Like a Little,Like a Little,Like a Lot,Like a Little,Like a Lot,Like a Lot,Like a Little,Like a Lot,Like a Little
11,Sometimes,Often,Rarely (once or twice),Rarely (once or twice),Often,Rarely (once or twice),Sometimes,Sometimes,Sometimes,Never,...,Like a Lot,Like a Lot,Love,Love,Love,Like a Lot,Love,Like a Little,Like a Little,Like a Little


/Users/nikkibytes/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


'[INFO] updated version: '

,w1behav_fci#1_1,w1behav_fci#1_2,w1behav_fci#1_3,w1behav_fci#1_4,w1behav_fci#1_5,w1behav_fci#1_6,w1behav_fci#1_7,w1behav_fci#1_8,w1behav_fci#1_9,w1behav_fci#1_10,...,w1behav_fci#2_19,w1behav_fci#2_20,w1behav_fci#2_21,w1behav_fci#2_22,w1behav_fci#2_23,w1behav_fci#2_24,w1behav_fci#2_25,w1behav_fci#2_26,w1behav_fci#2_27,w1behav_fci#2_28
7,2,3,3,1,2,3,3,1,2,3,...,2,2,2,2,2,2,2,4,2,1
8,3,3,1,1,1,1,1,2,1,2,...,3,1,3,1,1,1,3,3,3,2
9,3,3,1,1,1,1,3,3,2,1,...,1,2,3,1,4,3,4,2,3,2
10,3,3,1,1,1,1,2,2,1,1,...,2,2,2,3,2,3,3,2,3,2
11,3,4,2,2,4,2,3,3,3,1,...,3,3,4,4,4,3,4,2,2,2


In [197]:
fci_calc =  var_score_df[var_score_df['Set'] == "FCI"]


In [203]:
## FCI Calculations

In [213]:
for index, row in fci_calc[0:1].iterrows():
    output= row["Output"]
    calc = row["Calculation"]
    #method = calc.split("(")[0]
      # starting calculation
    if "MEAN" in calc:
        final_avg=0
        
        total_sum = 0 
        # stripped away the SUM and "()" 
        variables = calc.strip("MEAN").strip("()")
        variables=[x.strip(" ") for x in variables.split(",") ]
        var_ct=len(variables)
        #print(var_ct)

        # loop through variables 
        for x in variables:
            var=x
            #print(index)
            # have target variable, find numeric value in df
            var_val = fci_df.loc[index, var]
            print("Adding variable, {}: {},  to {} ".format(var,var_val, total_sum))
            #print(fci_df.loc[index, :])
            total_sum += int(var_val)
            
        print("TOTAL SUM: ", total_sum)
        print("MEAN: ", total_sum/var_ct)

Adding variable, w1behav_fci#1_3: 1,  to 0 
Adding variable, w1behav_fci#1_6: 1,  to 1 
Adding variable, w1behav_fci#1_4: 1,  to 2 
Adding variable, w1behav_fci#1_15: 1,  to 3 
Adding variable, w1behav_fci#1_26: 1,  to 4 
Adding variable, w1behav_fci#1_19: 3,  to 5 
Adding variable, w1behav_fci#1_10: 1,  to 8 
Adding variable, w1behav_fci#1_27: 1,  to 9 
TOTAL SUM:  10
MEAN:  1.25
